<div class="alert alert-block alert-info">
⚠️All notebooks should start with the following cell ⚠️.
<br>
<br>
This will ensure that the notebook is set in the correct working directory and that all internal modules can be loaded without setting an entry point.
</div>

In [79]:
from utils import setup_env_path

setup_env_path()

%load_ext autoreload
%autoreload 2

Former working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs
Current working directory:  /Users/ambroisebertin/Desktop/prog/prog_abeilles/fil-rouge-pollinisateurs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<div class="alert alert-block alert-danger">
Put your API key in the following cell.<br><br>
⚠️But DO NOT commit it to the repository, as it is a secret.⚠️
</div>

In [80]:
API_keys = [
    "2b10Me1HF0rfjoGWCseolNa8e",
    "2b10Xs7brWPuBdRTSeWx9V7HJu",
    "2b10umm8L2jTYjlWrJKPfshJ0O",
    "2b10sEg5pSsrtVT372XqlrzcLe",
    "2b10LDWWlXJwuS6FleaqfK6Rke",
]

# Plantnet use case - Data quality

## Data import 

In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [139]:
df = pd.read_csv('data/spipoll.csv')

/var/folders/cj/gd9mq5b53w51g6wvzmjj8ymr0000gn/T/ipykernel_67237/1749804913.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/spipoll.csv')


## Data aggregation

In [140]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]


df_filtered = df.loc[
    (
        (df["plante_sc"].isna() | df["plante_sc"].isin(values_to_check))
        & (df["plante_fr"].isna() | df["plante_fr"].isin(values_to_check))
        & (df["plante_precision"].isna() | df["plante_precision"].isin(values_to_check))
    )
    | ((df["plante_inconnue"] == 1.0))
]
df_filtered.shape

(18249, 31)

In [141]:
# group by collection_id and count the number of rows
missing_plants = (df_filtered
                    .drop_duplicates(subset='collection_id', keep='first')
                    )
missing_plants.shape
print(f"{(df_filtered.shape[0]/df.shape[0]*100):.2f} % of the data is missing all plant information")

2.72 % of the data is missing all plant information


In [142]:
missing_plants.shape

(2544, 31)

## Get data from API

In [143]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor
predictor = PlantNetPredictor(key=API_keys[0])

In [144]:
missing_plants['photo_fleur'].iloc[0]

'https://spgp-api.65mo.fr/api/containers/spgp/download/1546962826333.jpg'

In [145]:
missing_plants.head()

,collection_id,protocole_long,user_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,...,insecte_fr,insecte_denominationPlusPrecise,insecte_CdNomtaxref,insecte_abondance,insecte_commentaire,insecte_vu_sur_fleur,nb_validation,nb_suggestion,latitude,longitude
46,14,NaN,11 784,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches difficiles à déterminer,NaN,NaN,1,NaN,0.0,3.0,NaN,44.694536,4.662742
364,117,NaN,11 784,Plante inconnue,NaN,NaN,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches aux reflets métalliques,NaN,NaN,1,NaN,0.0,3.0,NaN,44.729200,4.574164
574,177,NaN,11 784,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,Les Mouches difficiles à déterminer,NaN,NaN,1,NaN,0.0,3.0,NaN,44.719394,4.598397
730,215,NaN,8 868,NaN,NaN,NaN,1.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,L'Abeille mellifère,NaN,NaN,1,NaN,0.0,3.0,NaN,42.674231,9.291175
731,216,NaN,8 868,NaN,NaN,NaN,1.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,...,L'Abeille mellifère,NaN,NaN,1,NaN,0.0,3.0,NaN,42.674242,9.291344


In [146]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor

predictor = PlantNetPredictor(key = API_keys[0])
prediction = None

try:
    prediction = predictor.predict(imageURL= missing_plants['photo_fleur'].iloc[-1], 
                    organs = "auto",
                    includeRelatedImages=False)
except Exception as e:
    print(f"Error: {e}")

In [147]:
prediction["results"][0]["species"]["family"]["scientificNameWithoutAuthor"]

'Asteraceae'

rappel : ordre > famille > genre > espèce

nous en plus général on veut la famille 'family' ici exemple Asteraceae 
nous en plus précis on veut le genre voire l'espèce (ici 'scientificNameWithoutAuthor')

ccl : on récupère 'family' > 'genus' > 'scientificNameWithoutAuthor'

In [148]:
prediction["results"][0]["species"]

{'scientificNameWithoutAuthor': 'Tolpis virgata',
 'scientificNameAuthorship': 'Bertol.',
 'genus': {'scientificNameWithoutAuthor': 'Tolpis',
  'scientificNameAuthorship': 'Adanson',
  'scientificName': 'Tolpis Adanson'},
 'family': {'scientificNameWithoutAuthor': 'Asteraceae',
  'scientificNameAuthorship': '',
  'scientificName': 'Asteraceae'},
 'commonNames': ['Trépane effilée'],
 'scientificName': 'Tolpis virgata Bertol.'}

In [149]:
prediction

{'query': {'project': 'weurope',
  'images': ['https://spgp-api.65mo.fr/api/containers/spgp/download/65a9ef74-7840-451b-bd93-8677a52adcd7.jpg'],
  'organs': ['auto'],
  'includeRelatedImages': False,
  'noReject': False},
 'language': 'fr',
 'preferedReferential': 'weurope',
 'results': [{'score': 0.14201,
   'species': {'scientificNameWithoutAuthor': 'Tolpis virgata',
    'scientificNameAuthorship': 'Bertol.',
    'genus': {'scientificNameWithoutAuthor': 'Tolpis',
     'scientificNameAuthorship': 'Adanson',
     'scientificName': 'Tolpis Adanson'},
    'family': {'scientificNameWithoutAuthor': 'Asteraceae',
     'scientificNameAuthorship': '',
     'scientificName': 'Asteraceae'},
    'commonNames': ['Trépane effilée'],
    'scientificName': 'Tolpis virgata Bertol.'},
   'gbif': {'id': '3097191'}},
  {'score': 0.1416,
   'species': {'scientificNameWithoutAuthor': 'Crepis micrantha',
    'scientificNameAuthorship': 'Czerep.',
    'genus': {'scientificNameWithoutAuthor': 'Crepis',
     

## Suite

### creation des datasets

In [107]:
# Import spipoll.csv as a pandas dataframe
spipoll = pd.read_csv("././data/spipoll.csv",low_memory=False)

# Extract relevant columns from the spipoll dataframe
plantes = spipoll[['collection_id', 'plante_sc', 'plante_fr',
       'plante_precision', 'plante_inconnue', 'plante_caractere',
       'photo_fleur', 'photo_plante', 'photo_feuille']]

# Shrinking the data by grouping by collection_id
# Keep only the first row for each unique value of collection_id
plantes = plantes.drop_duplicates(subset='collection_id', keep='first')

In [108]:
# add three columns to the dataframe named "group"
plantes['group'] = np.nan

In [109]:
plantes.shape

(75372, 10)

#### description des 3 différents datsets à extraire

1. plante_sc et qui ne contient pas NA ou données vides dans plante_sc
2. plante_fr et qui ne contient pas NA ou données vides dans plante_fr
3. le reste (plantes -1 -2)(qui normalement doit être de la même taille que missing_plants dans l'idéal)

quantifier la taille des différents df

In [110]:
values_to_check = ["Je ne sais pas", "Plante inconnue"]

### datset 1

In [124]:
# creation du 1er dataframe decrit ci dessus

plantes_to_remove_sc = plantes.loc[ #renommer en plantes_unlabelled_sc pour plus de clarté ??
    (
        (plantes["plante_sc"].isna() | plantes["plante_sc"].isin(values_to_check))
    )
    | ((plantes["plante_inconnue"] == 1.0))
]

# creation du dataframe contenant les valeurs de plantes mais pas de plantes_to_remove_sc
plantes_filtered_sc = plantes[~plantes.index.isin(plantes_to_remove_sc.index)]

# in plante add "1" to the column group for each row that is in plantes_filtered_sc
plantes.loc[plantes_filtered_sc.index, 'group'] = 1

plantes_filtered_sc.shape

(63465, 10)

In [125]:
plantes_to_remove_sc.shape

(11907, 10)

In [126]:
plantes_to_remove_sc.head()

,collection_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group
39,11,Plante inconnue,NaN,Mahonia faux-houx (Berberis aquifolium),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
46,14,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
78,25,Plante inconnue,NaN,Camélia du Japon (Camellia japonica),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
96,32,Plante inconnue,NaN,Jasmin d'hiver (Jasminum nudiflorum),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
168,62,Plante inconnue,NaN,polygala myrtifolia,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN


In [127]:
plantes.head()

,collection_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group
0,1,Les Trèfles à fleurs blanches ou roses en boul...,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
2,2,Acanthus mollis,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0


In [128]:
# affiche plantes.head où group != 1
plantes[plantes['group'] != 1].head()

,collection_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group
39,11,Plante inconnue,NaN,Mahonia faux-houx (Berberis aquifolium),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
46,14,Plante inconnue,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
78,25,Plante inconnue,NaN,Camélia du Japon (Camellia japonica),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
96,32,Plante inconnue,NaN,Jasmin d'hiver (Jasminum nudiflorum),0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN
168,62,Plante inconnue,NaN,polygala myrtifolia,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,NaN


In [129]:
# compte plantes où group != 1
plantes[plantes['group'] != 1].shape

(11907, 10)

### dataset 2

In [130]:
# creation du 2eme dataframe decrit ci dessus

plantes_to_remove_fr = plantes_to_remove_sc.loc[
    (
       (plantes_to_remove_sc["plante_fr"].isna() | plantes_to_remove_sc["plante_fr"].isin(values_to_check))
    )
    | ((plantes_to_remove_sc["plante_inconnue"] == 1.0))
]

# creation du dataframe contenant les valeurs de plantes_sc vides étiquettées dans plantes_fr
plantes_filtered_fr = plantes_to_remove_sc[~plantes_to_remove_sc.index.isin(plantes_to_remove_fr.index)]

# in plante add "1" to the column group for each row that is in plantes_filtered_sc
plantes.loc[plantes_filtered_sc.index, 'group'] = 1

plantes_filtered_fr.shape

(0, 10)

In [131]:
plantes_to_remove_fr.shape

(11907, 10)

il semblerait que toutes les lignes qui contiennent de l'info utile sur plante_fr soient egalement decrites par plante_sc

In [132]:
# display the rows of plantes_to_remove_sc where plant_fr is not null and not in values_to_check
plantes_to_remove_sc.loc[~(plantes_to_remove_sc["plante_fr"].isna() | plantes_to_remove_sc["plante_fr"].isin(values_to_check))]

,collection_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group


C'est bien le cas d'après cette ligne de code également.

### restructuration des deux datasets

On ne garde donc que les deux df plantes_filtered_sc et plantes_to_remove_sc. On va les renommer plus explicitement et grouper les plantes étiquettées par valeurs uniques.

In [133]:
plantes_sc_labelled = plantes_filtered_sc
plantes_sc_unlabelled = plantes_to_remove_sc

# verification des dimensions des deux df par rapport à la dim de plantes
plantes_sc_labelled.shape[0] + plantes_sc_unlabelled.shape[0] == plantes.shape[0]

True

In [134]:
# group plantes_sc_labelled by unique values of plante_sc
plantes_sc_labelled = plantes_sc_labelled.drop_duplicates(subset='plante_sc', keep='first')

# display the dimension of the dataframe
plantes_sc_labelled.shape

(2620, 10)

On a donc un df avec 2620 plantes dont le nom est spécifié dans plante_sc et 11.907 (~1.8% des données d'origine, spipoll.csv) dont le nom n'est pas spécifié dans plante_sc.

In [136]:
plantes_sc_unlabelled.shape[0] / df.shape[0]

0.017751929201006645

## Suite du projet : appel à l'API

### appel à l'API

In [150]:
from data_quality.plant_treatment.plantnet_api import PlantNetPredictor

predictor = PlantNetPredictor(key = API_keys[0])
prediction = None

try:
    prediction = predictor.predict(imageURL= plantes_sc_labelled['photo_fleur'].iloc[0], # modifier cette ligne pour qu'elle traite chaque ligne de plantes_sc_labelled
                    organs = "auto",
                    includeRelatedImages=False)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
for row in plantes_sc_labelled.iterrows(): #remplacer df par le nom du dataframe voulu
    print(row)
    break

In [151]:
plantes_sc_labelled.head()

,collection_id,plante_sc,plante_fr,plante_precision,plante_inconnue,plante_caractere,photo_fleur,photo_plante,photo_feuille,group
0,1,Les Trèfles à fleurs blanches ou roses en boul...,NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
2,2,Acanthus mollis,Acanthe molle,NaN,NaN,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
5,3,"Les Orchidées à fleurs blanches, jaunes ou ver...",NaN,orchis bouc - Himantoglossum hircinum,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
6,4,Les Vipérines à fleurs bleues-violacées (des e...,NaN,Theaceae,0.0,plantée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0
10,5,Les Pâquerettes (Bellis sp),NaN,NaN,0.0,spontanée,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,https://spgp-api.65mo.fr/api/containers/spgp/d...,1.0


In [162]:
prediction["results"][0]["species"]

{'scientificNameWithoutAuthor': 'Trifolium pratense',
 'scientificNameAuthorship': 'L.',
 'genus': {'scientificNameWithoutAuthor': 'Trifolium',
  'scientificNameAuthorship': 'L.',
  'scientificName': 'Trifolium L.'},
 'family': {'scientificNameWithoutAuthor': 'Fabaceae',
  'scientificNameAuthorship': '',
  'scientificName': 'Fabaceae'},
 'commonNames': ['Trèfle des prés', 'Trèfle rouge', 'Trèfle commun'],
 'scientificName': 'Trifolium pratense L.'}

rappel : ordre > famille > genre > espèce

on récupère tout sauf l'ordre

In [168]:
prediction["results"][0]["species"]["family"]["scientificNameWithoutAuthor"] # famille

'Fabaceae'

In [169]:
prediction["results"][0]["species"]["genus"]["scientificNameWithoutAuthor"] # genre

'Trifolium'

In [170]:
prediction["results"][0]["species"]["scientificNameWithoutAuthor"] # espece

'Trifolium pratense'